## Objective:
    
To make a theory calculation for the 2D (angular) cross correlation for the PanSTARRS Supernova and BOSS (CMASS & LOWZ) galaxy data sets, that was found previously for comparison and validation.

### Notes: 

- NIntegrate Chi integral in Mathematica

Author: Megan Tabbutt <br/>
Last Revised: 8_7_20

### Calculations:

_(2D) Cross Correlation Function (from Ross):_

$w(\theta)^{gg}_{ur} = b_u b_r \; \int \; n_u(z) n_r(z) \; \frac{dz}{d\chi} \frac{dz}{d\chi} \; d\chi \; \int \frac{k}{2 \pi} \; P(k, z(\chi)) \; J_0(\chi k \theta) \;dk $

Comoving distance formula from Dodelson:

$\chi(a) = \int^1_a \frac{da}{a^2H(a)}$ with $a = \frac{1}{1+z}$, $\chi(z) = \int^z_0 \frac{dz}{H(z)}$

and 

$H(z)^2 = H_0^2(\Omega_m(1+z)^3 + \Omega_k(1+z)^2 + \Omega_{\Lambda})$ (Wikipedia - Hubble Law)

In [26]:
# Working on 2015 laptop: Need to be in CCL conda environment to work on CAMB
import camb
import numpy as np
import pandas as pd
import sqlite3

In [9]:
DATA_PATH = '/Users/megantabbutt/CosmologyDataProducts/'

### Pull in and Parse Data:

In [11]:
qry_PanSTARRS_Data_All = "SELECT ID, DEC, RA, zSN, zHost FROM PanSTARRSNEW WHERE (zSN > -999) || (zHost > -999)"

connPAN = sqlite3.connect(DATA_PATH + 'PanSTARRS.db')
PanSTARRSNEW_GoodZ = pd.read_sql(qry_PanSTARRS_Data_All, con=connPAN)
connPAN.close()
PanSTARRSNEW_GoodZ.head(3) # 1129 objects over 10 pointings 

,ID,DEC,RA,zSN,zHost
0,ps0909006,1.184831,333.9500,0.284,-999.0
1,ps0909010,-4.078911,37.1182,0.270,-999.0
2,ps0910017,-5.023289,35.2775,0.320,-999.0


##### Note: Take the redshift of the supernova if both are given. It redshift of supernova is unknown, take the redshift of the host. 

In [31]:
PanSTARRSNEW_GoodZ['zBest'] = np.nan

j = 0
for i in PanSTARRSNEW_GoodZ.iterrows():
    
    if i[1]['zSN'] >= 0:
        print(PanSTARRSNEW_GoodZ[j])
        i[1]['zBest'] = i[1]['zSN']
    else:
        i[1]['zBest'] = i[1]['zHost']
    
    j += 1
        
PanSTARRSNEW_GoodZ

KeyError: 0

In [ ]:

for i in PanSTARRSNEW_GoodZ.iterrows():
    print("______")
    print(i[1]['zSN'])
    print(i[1]['zHost'])
    if i[1]['zSN'] >= 0:
        print('Supernova:')
        print(i[1]['zSN'])
    else:
        print('Host:')
        print(i[1]['zHost'])

### Get chi(z) and interpolate to get z(chi):